# Projet D - RAG : Question-Answering sur Documents

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Construire un système de question-answering avec RAG

---

## Objectifs du projet

Dans ce projet, vous allez :
1. Comprendre le principe du RAG (Retrieval-Augmented Generation)
2. Créer un index de documents avec des embeddings
3. Implémenter la recherche sémantique
4. Combiner retrieval + génération pour répondre à des questions

## 0. Installation

In [ ]:
!pip install torch transformers sentence-transformers matplotlib numpy tqdm faiss-cpu -q

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

---

## 1. Pourquoi le RAG ?

### Le problème des LLMs

Les modèles de langage (GPT, BERT...) ont des **limitations** :

| Limitation | Exemple |
|------------|--------|
| **Connaissance figée** | Ne connaît pas les événements après son entraînement |
| **Hallucinations** | Invente des faits avec confiance |
| **Pas de sources** | Ne peut pas citer d'où vient l'information |
| **Données privées** | Ne connaît pas vos documents internes |

### La solution : RAG

**RAG** = Retrieval-Augmented Generation

```
Question: "Quelle est la politique de congés de l'entreprise ?"
        ↓
    [RETRIEVER]
    Cherche dans la base documentaire
        ↓
    Documents pertinents trouvés:
    - "Article 5.2: Les employés ont droit à 25 jours..."
    - "Note RH: Les congés doivent être posés 2 semaines avant..."
        ↓
    [GENERATOR]
    LLM génère une réponse basée sur les documents
        ↓
    Réponse: "Selon l'article 5.2, vous avez droit à 25 jours de congés.
              Les congés doivent être posés 2 semaines à l'avance."
```

### Architecture RAG

```
┌─────────────────────────────────────────────────────────────┐
│                         RAG SYSTEM                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Question ──► [Encoder] ──► Query Embedding                 │
│                                    │                        │
│                                    ▼                        │
│              ┌─────────────────────────────┐                │
│              │     INDEX VECTORIEL         │                │
│              │  (embeddings des documents) │                │
│              └─────────────────────────────┘                │
│                                    │                        │
│                                    ▼                        │
│              Top-K documents les plus similaires            │
│                                    │                        │
│                                    ▼                        │
│  ┌─────────────────────────────────────────────────┐        │
│  │ Prompt = Question + Contexte (documents trouvés)│        │
│  └─────────────────────────────────────────────────┘        │
│                                    │                        │
│                                    ▼                        │
│                              [LLM / Generator]              │
│                                    │                        │
│                                    ▼                        │
│                               Réponse                       │
└─────────────────────────────────────────────────────────────┘
```

---

## 2. Préparation des Documents

On va créer une base de connaissances sur les **Transformers** (le sujet de ce cours !).

In [ ]:
# Base de connaissances (documents)
documents = [
    # Architecture
    {
        "id": 1,
        "title": "Architecture Transformer",
        "content": "Le Transformer est une architecture de réseau de neurones introduite en 2017 dans le papier 'Attention Is All You Need'. Il se compose d'un encodeur et d'un décodeur, tous deux basés sur le mécanisme d'attention. L'encodeur traite la séquence d'entrée en parallèle, tandis que le décodeur génère la séquence de sortie de manière autoregressive."
    },
    {
        "id": 2,
        "title": "Mécanisme d'Attention",
        "content": "L'attention permet à chaque position de la séquence de 'regarder' toutes les autres positions. Elle utilise trois vecteurs : Query (Q), Key (K) et Value (V). Le score d'attention est calculé par le produit scalaire Q·K, normalisé par la racine de la dimension, puis passé dans un softmax. La formule est : Attention(Q,K,V) = softmax(QK^T/√d_k)V."
    },
    {
        "id": 3,
        "title": "Multi-Head Attention",
        "content": "Le Multi-Head Attention utilise plusieurs 'têtes' d'attention en parallèle. Chaque tête peut apprendre à détecter différents types de relations : syntaxiques, sémantiques, de proximité, etc. Les sorties des têtes sont concaténées puis projetées. BERT-base utilise 12 têtes, GPT-3 en utilise 96."
    },
    {
        "id": 4,
        "title": "Positional Encoding",
        "content": "Comme le Transformer traite les tokens en parallèle, il n'a pas de notion d'ordre. Le Positional Encoding ajoute une information de position à chaque embedding. La version originale utilise des fonctions sinusoïdales : PE(pos,2i) = sin(pos/10000^(2i/d)) et PE(pos,2i+1) = cos(pos/10000^(2i/d)). Les modèles récents utilisent souvent des embeddings de position appris."
    },
    # Modèles
    {
        "id": 5,
        "title": "BERT",
        "content": "BERT (Bidirectional Encoder Representations from Transformers) est un modèle créé par Google en 2018. Il utilise uniquement l'encodeur du Transformer et est pré-entraîné avec deux tâches : Masked Language Modeling (MLM) où 15% des tokens sont masqués, et Next Sentence Prediction (NSP). BERT-base a 110M de paramètres et 12 couches."
    },
    {
        "id": 6,
        "title": "GPT",
        "content": "GPT (Generative Pre-trained Transformer) est développé par OpenAI. Contrairement à BERT, GPT utilise uniquement le décodeur avec un masque causal : chaque token ne peut voir que les tokens précédents. GPT est entraîné à prédire le token suivant. GPT-3 a 175 milliards de paramètres et GPT-4 est estimé à plus de 1000 milliards."
    },
    {
        "id": 7,
        "title": "T5",
        "content": "T5 (Text-to-Text Transfer Transformer) de Google reformule toutes les tâches NLP en génération de texte. Par exemple, la classification devient : 'classify: This movie is great' → 'positive'. T5 utilise l'architecture encodeur-décodeur complète. Le modèle T5-11B a 11 milliards de paramètres."
    },
    # Entraînement
    {
        "id": 8,
        "title": "Pré-entraînement",
        "content": "Le pré-entraînement consiste à entraîner un modèle sur une grande quantité de texte non annoté avec des tâches auto-supervisées. Pour BERT c'est le MLM, pour GPT c'est la prédiction du token suivant. Cette phase capture les connaissances linguistiques générales et nécessite d'énormes ressources de calcul."
    },
    {
        "id": 9,
        "title": "Fine-tuning",
        "content": "Le fine-tuning adapte un modèle pré-entraîné à une tâche spécifique. On ajoute une tête de classification et on entraîne sur un dataset annoté. Le fine-tuning nécessite beaucoup moins de données et de calcul que le pré-entraînement. Typiquement quelques heures sur un GPU contre des semaines pour le pré-entraînement."
    },
    {
        "id": 10,
        "title": "Tokenization",
        "content": "La tokenization découpe le texte en unités (tokens). Les méthodes modernes comme BPE (Byte Pair Encoding) ou WordPiece créent un vocabulaire de sous-mots. Par exemple 'unhappiness' devient ['un', '##happy', '##ness']. Cela permet de gérer les mots rares tout en gardant un vocabulaire de taille raisonnable (30k-50k tokens)."
    },
    # Applications
    {
        "id": 11,
        "title": "Classification de texte",
        "content": "Pour la classification avec BERT, on utilise le token [CLS] qui résume toute la séquence. On ajoute une couche linéaire qui projette l'embedding [CLS] vers le nombre de classes. Pour GPT, on peut utiliser le dernier token ou faire une moyenne des embeddings."
    },
    {
        "id": 12,
        "title": "Question Answering",
        "content": "En Question Answering extractif, le modèle doit trouver la réponse dans un contexte donné. Le modèle prédit deux positions : le début et la fin de la réponse dans le texte. On entraîne avec des datasets comme SQuAD qui contient 100k+ paires question-réponse avec leur contexte."
    },
    {
        "id": 13,
        "title": "Génération de texte",
        "content": "La génération de texte avec GPT est autoregressive : on prédit un token, on l'ajoute à l'entrée, puis on prédit le suivant. La température contrôle la 'créativité' : température basse = plus déterministe, température haute = plus aléatoire. Le top-k et top-p (nucleus sampling) limitent les tokens candidats."
    },
    # Concepts avancés
    {
        "id": 14,
        "title": "RAG",
        "content": "RAG (Retrieval-Augmented Generation) combine recherche documentaire et génération. Au lieu de tout stocker dans les paramètres du modèle, on cherche les documents pertinents dans une base externe puis on les donne au LLM comme contexte. Cela réduit les hallucinations et permet de citer les sources."
    },
    {
        "id": 15,
        "title": "Embeddings",
        "content": "Les embeddings sont des représentations vectorielles denses. Les mots similaires ont des embeddings proches dans l'espace vectoriel. Sentence-BERT produit des embeddings de phrases entières, utiles pour la recherche sémantique. La similarité cosinus mesure la proximité entre deux embeddings."
    },
]

print(f"Base de connaissances: {len(documents)} documents")
for doc in documents[:3]:
    print(f"  - {doc['title']}")

---

## 3. Création des Embeddings

On utilise **Sentence-BERT** pour créer des embeddings de documents.

In [ ]:
# Charger le modèle d'embeddings
print("Chargement du modèle d'embeddings...")
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print(f"Modèle chargé ! Dimension des embeddings: {embedding_model.get_sentence_embedding_dimension()}")

In [ ]:
# ============================================
# EXERCICE 1 : Créer les embeddings des documents
# ============================================

def create_document_embeddings(documents, model):
    """
    Crée les embeddings pour chaque document.
    
    Args:
        documents: Liste de dicts avec 'title' et 'content'
        model: Modèle SentenceTransformer
    
    Returns:
        embeddings: np.array de shape (n_docs, embed_dim)
    """
    # TODO: Créer les embeddings
    # On combine title + content pour chaque document
    
    texts = []  # Liste des textes à encoder
    for doc in documents:
        # Combiner titre et contenu
        text = f"{doc['title']}: {doc['content']}"
        texts.append(text)
    
    # Encoder avec le modèle
    embeddings = model.encode(texts, show_progress_bar=True)
    
    return np.array(embeddings)

# Créer les embeddings
doc_embeddings = create_document_embeddings(documents, embedding_model)
print(f"Embeddings shape: {doc_embeddings.shape}")

---

## 4. Index Vectoriel avec FAISS

**FAISS** (Facebook AI Similarity Search) permet de faire des recherches rapides dans de grands ensembles de vecteurs.

In [ ]:
# ============================================
# EXERCICE 2 : Créer l'index FAISS
# ============================================

def create_faiss_index(embeddings):
    """
    Crée un index FAISS pour la recherche de similarité.
    
    Args:
        embeddings: np.array de shape (n_docs, embed_dim)
    
    Returns:
        index: Index FAISS
    """
    # TODO: Créer l'index
    
    # Dimension des embeddings
    dim = embeddings.shape[1]
    
    # Créer un index avec similarité cosinus
    # On normalise les vecteurs et on utilise IndexFlatIP (Inner Product)
    
    # Normaliser les embeddings (pour que IP = cosine similarity)
    faiss.normalize_L2(embeddings)
    
    # Créer l'index
    index = faiss.IndexFlatIP(dim)  # IP = Inner Product
    
    # Ajouter les vecteurs
    index.add(embeddings)
    
    return index

# Créer l'index
index = create_faiss_index(doc_embeddings.copy())  # copy car normalize_L2 modifie in-place
print(f"Index créé avec {index.ntotal} vecteurs")

In [ ]:
# Test de recherche
def search(query, model, index, documents, top_k=3):
    """
    Recherche les documents les plus similaires à une requête.
    
    Args:
        query: Texte de la requête
        model: Modèle d'embeddings
        index: Index FAISS
        documents: Liste des documents
        top_k: Nombre de résultats
    
    Returns:
        Liste de (document, score)
    """
    # Encoder la requête
    query_embedding = model.encode([query])
    
    # Normaliser
    faiss.normalize_L2(query_embedding)
    
    # Rechercher
    scores, indices = index.search(query_embedding, top_k)
    
    # Retourner les résultats
    results = []
    for score, idx in zip(scores[0], indices[0]):
        results.append((documents[idx], score))
    
    return results

# Test
query = "Comment fonctionne l'attention dans les Transformers ?"
results = search(query, embedding_model, index, documents, top_k=3)

print(f"Requête: '{query}'\n")
print("Résultats:")
for doc, score in results:
    print(f"  [{score:.3f}] {doc['title']}")
    print(f"           {doc['content'][:100]}...\n")

---

## 5. Génération de Réponses

On combine maintenant retrieval + génération.

In [ ]:
# Charger un modèle de génération
print("Chargement du modèle de génération...")
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device=0 if torch.cuda.is_available() else -1
)
print("Modèle chargé !")

In [ ]:
# ============================================
# EXERCICE 3 : Implémenter le RAG complet
# ============================================

class SimpleRAG:
    """
    Système RAG simple pour question-answering.
    """
    
    def __init__(self, documents, embedding_model, generator, top_k=3):
        self.documents = documents
        self.embedding_model = embedding_model
        self.generator = generator
        self.top_k = top_k
        
        # Créer les embeddings et l'index
        print("Création de l'index...")
        self.embeddings = self._create_embeddings()
        self.index = self._create_index()
        print(f"Index prêt avec {len(documents)} documents.")
    
    def _create_embeddings(self):
        texts = [f"{doc['title']}: {doc['content']}" for doc in self.documents]
        return self.embedding_model.encode(texts)
    
    def _create_index(self):
        embeddings = self.embeddings.copy()
        faiss.normalize_L2(embeddings)
        index = faiss.IndexFlatIP(embeddings.shape[1])
        index.add(embeddings)
        return index
    
    def retrieve(self, query):
        """Récupère les documents pertinents."""
        query_emb = self.embedding_model.encode([query])
        faiss.normalize_L2(query_emb)
        scores, indices = self.index.search(query_emb, self.top_k)
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            results.append({
                'document': self.documents[idx],
                'score': score
            })
        return results
    
    def generate_answer(self, query, context_docs):
        """Génère une réponse basée sur le contexte."""
        
        # TODO: Construire le prompt
        # Format: "Contexte: ... Question: ... Réponse:"
        
        # Construire le contexte
        context_parts = []
        for doc_info in context_docs:
            doc = doc_info['document']
            context_parts.append(f"- {doc['title']}: {doc['content']}")
        
        context = "\n".join(context_parts)
        
        # Construire le prompt
        prompt = f"""Réponds à la question en utilisant uniquement les informations du contexte.

Contexte:
{context}

Question: {query}

Réponse:"""
        
        # Générer
        response = self.generator(
            prompt,
            max_length=200,
            num_return_sequences=1
        )[0]['generated_text']
        
        return response
    
    def answer(self, query, verbose=True):
        """
        Pipeline complet : retrieve + generate.
        
        Args:
            query: Question de l'utilisateur
            verbose: Afficher les détails
        
        Returns:
            dict avec 'answer', 'sources', 'query'
        """
        # 1. Retrieve
        retrieved = self.retrieve(query)
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"Question: {query}")
            print(f"{'='*60}")
            print(f"\nDocuments trouvés:")
            for r in retrieved:
                print(f"  [{r['score']:.3f}] {r['document']['title']}")
        
        # 2. Generate
        answer = self.generate_answer(query, retrieved)
        
        if verbose:
            print(f"\nRéponse: {answer}")
            print(f"{'='*60}\n")
        
        return {
            'query': query,
            'answer': answer,
            'sources': [r['document']['title'] for r in retrieved]
        }

In [ ]:
# Créer le système RAG
rag = SimpleRAG(
    documents=documents,
    embedding_model=embedding_model,
    generator=generator,
    top_k=3
)

In [ ]:
# Tests
questions = [
    "Qu'est-ce que l'attention dans les Transformers ?",
    "Quelle est la différence entre BERT et GPT ?",
    "Comment fonctionne le fine-tuning ?",
    "Qu'est-ce que le RAG et pourquoi est-ce utile ?",
    "Combien de paramètres a GPT-3 ?",
]

for q in questions:
    result = rag.answer(q)

---

## 6. Évaluation et Analyse

In [ ]:
# Visualiser la similarité entre documents
from sklearn.metrics.pairwise import cosine_similarity

# Calculer la matrice de similarité
sim_matrix = cosine_similarity(doc_embeddings)

# Visualiser
plt.figure(figsize=(12, 10))
plt.imshow(sim_matrix, cmap='Blues')
plt.colorbar(label='Similarité cosinus')

# Labels
titles = [doc['title'][:15] + '...' if len(doc['title']) > 15 else doc['title'] 
          for doc in documents]
plt.xticks(range(len(documents)), titles, rotation=45, ha='right', fontsize=8)
plt.yticks(range(len(documents)), titles, fontsize=8)

plt.title('Similarité entre documents')
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation t-SNE des documents
from sklearn.manifold import TSNE

# Réduire en 2D
tsne = TSNE(n_components=2, perplexity=5, random_state=42)
embeddings_2d = tsne.fit_transform(doc_embeddings)

# Catégories manuelles pour la couleur
categories = {
    'Architecture': [0, 1, 2, 3],
    'Modèles': [4, 5, 6],
    'Entraînement': [7, 8, 9],
    'Applications': [10, 11, 12, 13, 14]
}

colors = ['blue', 'green', 'orange', 'red']
color_map = {}
for i, (cat, indices) in enumerate(categories.items()):
    for idx in indices:
        color_map[idx] = colors[i]

# Plot
plt.figure(figsize=(12, 8))

for i, (x, y) in enumerate(embeddings_2d):
    plt.scatter(x, y, c=color_map.get(i, 'gray'), s=100)
    plt.annotate(documents[i]['title'][:20], (x, y), fontsize=8)

# Légende
for cat, color in zip(categories.keys(), colors):
    plt.scatter([], [], c=color, label=cat)
plt.legend()

plt.title('Documents dans l\'espace des embeddings (t-SNE)')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.tight_layout()
plt.show()

---

## 7. Améliorations possibles

In [ ]:
# ============================================
# EXERCICE 4 : Ajouter le chunking
# ============================================

def chunk_document(doc, chunk_size=200, overlap=50):
    """
    Découpe un document en chunks plus petits.
    
    Args:
        doc: Dict avec 'content'
        chunk_size: Taille max d'un chunk (en mots)
        overlap: Chevauchement entre chunks
    
    Returns:
        Liste de chunks (dicts)
    """
    # TODO: Implémenter le chunking
    
    words = doc['content'].split()
    chunks = []
    
    for i in range(0, len(words), chunk_size - overlap):
        chunk_words = words[i:i + chunk_size]
        if len(chunk_words) < 20:  # Ignorer les chunks trop petits
            continue
            
        chunk = {
            'id': f"{doc['id']}_chunk_{len(chunks)}",
            'title': doc['title'],
            'content': ' '.join(chunk_words),
            'parent_id': doc['id']
        }
        chunks.append(chunk)
    
    # Si le document est petit, le garder tel quel
    if len(chunks) == 0:
        chunks.append(doc)
    
    return chunks

# Test
test_doc = documents[0]
chunks = chunk_document(test_doc, chunk_size=30, overlap=10)
print(f"Document original: {len(test_doc['content'].split())} mots")
print(f"Chunks créés: {len(chunks)}")
for c in chunks:
    print(f"  - {len(c['content'].split())} mots: {c['content'][:50]}...")

In [ ]:
# Interface interactive
def demo_rag():
    print("\n" + "="*50)
    print("RAG - Question Answering sur les Transformers")
    print("="*50)
    print("Posez vos questions sur les Transformers !")
    print("Tapez 'quit' pour quitter")
    print("="*50)
    
    while True:
        query = input("\nQuestion: ")
        if query.lower() == 'quit':
            break
        
        result = rag.answer(query)

# Décommenter pour tester
# demo_rag()

---

## 8. Récapitulatif

### Ce que vous avez appris

1. **RAG** : Combiner recherche documentaire et génération
2. **Embeddings** : Représenter textes et requêtes dans le même espace
3. **FAISS** : Index vectoriel pour recherche rapide
4. **Pipeline** : Retrieve → Augment → Generate

### Architecture RAG en production

| Composant | Options populaires |
|-----------|--------------------|
| Embeddings | OpenAI, Cohere, Sentence-BERT |
| Vector DB | Pinecone, Weaviate, Chroma, FAISS |
| LLM | GPT-4, Claude, Llama, Mistral |
| Framework | LangChain, LlamaIndex |

### Améliorations possibles

- **Chunking intelligent** : Découper par paragraphes/sections
- **Re-ranking** : Réordonner les résultats avec un modèle cross-encoder
- **Hybrid search** : Combiner recherche sémantique et lexicale (BM25)
- **Query expansion** : Reformuler la question pour améliorer le recall

### Limitations

- Qualité dépend de la base documentaire
- Latence ajoutée par la recherche
- Le LLM peut mal interpréter le contexte
- Pas de raisonnement multi-étapes complexe

In [ ]:
# Espace pour vos expérimentations

# Ajoutez vos propres documents !
# nouveau_doc = {
#     "id": 100,
#     "title": "Mon sujet",
#     "content": "..."
# }

# Essayez différentes questions
ma_question = "Comment BERT est-il entraîné ?"
result = rag.answer(ma_question)